# Retrieval Augmented Generation

![](https://www.dailydoseofds.com/content/images/2024/10/rag.gif)
Source: [Daily Dose of DS](https://www.dailydoseofds.com/a-crash-course-on-building-rag-systems-part-1-with-implementations/)

Overview:
- Introduction to RAG with LlamaIndex
- Data ingestion
  - PDF
  - Web pages
  - Code
- Data splitting
  - Token splitting
  - Sentence splitting
  - Structured data splitting
  - Semantic chunking
- Vectorization
  - Embeddings
  - Vector storage
- Retrieval
  - Keyword search
  - Vector search
  - Hybrid search
- Advanced methods
  - Query rewriting
  - Multi-hop retrieval

In [110]:
%load_ext rich
%load_ext autoreload
%autoreload 2


The rich extension is already loaded. To reload it, use:
  %reload_ext rich


# Introduction to RAG with LlamaIndex

LlamaIndex is a library for working with large language models.
One of its main strengths is its ability to ingest documents into a vector index and use them to answer questions.
This is known as Retrieval Augmented Generation (RAG).

To start, we will use a low-code, high-level abstraction to build a basic PDF question-answering system.
We will read in PDFs, split them into chunks, embed them, and store them in a vector database.
Then, we will use an abstraction known as a `QueryEngine` that implements RAG to answer questions about the documents.

In [4]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install llama-index \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-retrievers-bm25 \
        llama-index-llms-gemini \
        docling \
        llama-index-readers-docling \
        llama-index-node-parser-docling \
        PyStemmer
        
except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")


Not in colab - using local environment variables.

In [5]:
import os
import requests

# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21783"
pdf_path = os.path.join(data_dir, "2407.21783.pdf")

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")


PDF already exists at data/2407.21783.pdf

The first thing we need to do is to load the data.
For general documents like PDFs, LlamaIndex provides a nice abstraction known as a `SimpleDirectoryReader` that can load data from a directory.
In the cell below, we use it to load the data from the `data` directory.

In [6]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader(data_dir).load_data()

Let's take a look at the first document.
Most prominently, we get the text of the document.
By default, we also get a lot of useful information, like the page number file name, file path, type, size, etc.
When we load lots of documents, this type of information becomes important to keep track of.

In [7]:
print(documents[0])

Document(
    id_='d2ffb7ab-de1c-4d12-b2f3-0f28f81ca4a9',
    embedding=None,
    metadata={
        'page_label': '1',
        'file_name': '2407.21783.pdf',
        'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
        'file_type': 'application/pdf',
        'file_size': 9832488,
        'creation_date': '2024-11-14',
        'last_modified_date': '2024-11-14'
    },
    excluded_embed_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    excluded_llm_metadata_keys=[
        'file_name',
        'file_type',
        'file_size',
        'creation_date',
        'last_modified_date',
        'last_accessed_date'
    ],
    relationships={},
    metadata_template='{key}: {value}',
    metadata_separator='\n',
    text='The Llama 3 Herd of Models\nLlama Team, AI @ Meta 1\n1A detailed contributor list can be found in the 
appendix of this paper.\nModern artificial intelligence (AI) systems are powered by foundation models. This paper 
presents a\nnew set of foundation models, called Llama 3. It is a herd of language models that natively 
support\nmultilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with\n405B 
parameters and a context window of up to 128K tokens. This paper presents an extensive\nempirical evaluation of 
Llama 3. We find that Llama 3 delivers comparable quality to leading language\nmodels such as GPT-4 on a plethora 
of tasks. We publicly release Llama 3, including pre-trained and\npost-trained versions of the 405B parameter 
language model and our Llama Guard 3 model for input\nand output safety. The paper also presents the results of 
experiments in which we integrate image,\nvideo, and speech capabilities into Llama 3 via a compositional approach.
We observe this approach\nperforms competitively with the state-of-the-art on image, video, and speech recognition 
tasks. The\nresulting models are not yet being broadly released as they are still under development.\nDate: July 
23, 2024\nWebsite: https://llama.meta.com/\n1 Introduction\nFoundation models are general models of language, 
vision, speech, and/or other modalities that are designed\nto support a large variety of AI tasks. They form the 
basis of many modern AI systems.\nThe development of modern foundation models consists of two main stages:(1) a 
pre-training stage in which\nthe model is trained at massive scale using straightforward tasks such as next-word 
prediction or captioning\nand (2) a post-training stage in which the model is tuned to follow instructions, align 
with human preferences,\nand improve specific capabilities (for example, coding and reasoning).\nIn this paper, we 
present a new set of foundation models for language, calledLlama 3. The Llama 3 Herd\nof models natively supports 
multilinguality, coding, reasoning, and tool usage. Our largest model is dense\nTransformer with 405B parameters, 
processing information in a context window of up to 128K tokens. Each\nmember of the herd is listed in Table 1. All
the results presented in this paper are for the Llama 3.1 models,\nwhich we will refer to as Llama 3 throughout for
brevity.\nWe believe there are three key levers in the development of high-quality foundation models: data, scale, 
and\nmanaging complexity. We seek to optimize for these three levers in our development process:\n• Data. Compared 
to prior versions of Llama (Touvron et al., 2023a,b), we improved both the quantity 
and\nqualityofthedataweuseforpre-trainingandpost-training. Theseimprovementsincludethedevelopment\nof more careful 
pre-processing and curation pipelines for pre-training data and the development of more\nrigorous quality assurance
and filtering approaches for post-training data. We pre-train Llama 3 on a\ncorpus of about 15T multilingual 
tokens, compared to 1.8T tokens for Llama 2.\n• Scale. We t

Now that we've loaded the data, we need to vectorize it.
We will use a combination of an embedding model and a vector database to store the vectors.
In the cell below, we use a HuggingFace embedding model to embed the documents.
We also use torch to determine the device to use for the embedding model (`mps` for Mac GPUs, `cuda` for Nvidia GPUs, and `cpu` otherwise).

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from torch.backends.mps import is_available as is_mps_available
from torch.cuda import is_available as is_cuda_available

if is_mps_available():
    device = "mps"
elif is_cuda_available():
    device = "cuda"
else:
    device = "cpu"

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", device=device)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

Finally, let's set up our RAG query engine.
If we want to perform simple question-answering, we can use the `as_query_engine` method.
If we want to perform chat with history, we can use the `as_chat_engine` method.
We can see both below 👇.


In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # set gpt-4o-mini as the default llm

query_engine = index.as_query_engine(llm=llm)
chat_engine = index.as_chat_engine(llm=llm)

Let's see how the query enine works.
We start by passing it a question, then it uses the retriever to find the most relevant documents.
Finally, it uses the LLM to answer the question.

In [10]:
response = query_engine.query("How many new Llama models models are mentioned in the paper?")

Below we see the response text.
But there's also some additional information that we can access, including the source nodes.
These are the nodes that the retriever used to answer the question.
We can see that the retriever found several nodes that are relevant to the question, then the LLM used at least one of them to answer the question.

In [11]:
print(response.response)

The paper mentions a total of three new Llama models: Llama 3 with 8B parameters, Llama 3 with 70B parameters, and 
Llama 3 with 405B parameters.

Taking a look at the first source node, we can see that it is a node that contains part of the document that is relevant to the question.
It also contains the `score`, which is the similarity between the question and the node.


In [12]:
print(response.source_nodes[0])

NodeWithScore(
    node=TextNode(
        id_='7dee73d7-f185-4dd0-84d9-1830c15d286d',
        embedding=None,
        metadata={
            'page_label': '2',
            'file_name': '2407.21783.pdf',
            'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
            'file_type': 'application/pdf',
            'file_size': 9832488,
            'creation_date': '2024-11-14',
            'last_modified_date': '2024-11-14'
        },
        excluded_embed_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        excluded_llm_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        relationships={
            <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                node_id='c2aa5e07-1c64-44f6-8701-f3df53873209',
                node_type='4',
                metadata={
                    'page_label': '2',
                    'file_name': '2407.21783.pdf',
                    'file_path': 
'/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
                    'file_type': 'application/pdf',
                    'file_size': 9832488,
                    'creation_date': '2024-11-14',
                    'last_modified_date': '2024-11-14'
                },
                hash='2ea90099062b1dd3bee8c1ae15c1a08e55026fc535e90e0d917ed1f6139bfd74'
            )
        },
        metadata_template='{key}: {value}',
        metadata_separator='\n',
        text='Finetuned Multilingual Long context Tool use Release\nLlama 3 8B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 8B 
Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3 70B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 70B Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3.1 
8B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 8B Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 70B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 70B 
Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 405B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 405B Instruct ✓ ✓ ✓ ✓ July 2024\nTable 1 
Overview of the Llama 3 Herd of models. All results in this paper are for the Llama 3.1 models.\nscaling laws for 
foundation models, our flagship model outperforms smaller models trained using the\nsame procedure. While our 
scaling laws suggest our flagship model is an approximately compute-optimal\nsize for our training budget, we also 
train our smaller models for much longer than is compute-optimal.\nThe resulting models perform better than 
compute-optimal models at the same inference budget. We\nuse the flagship model to further improve the quality of 
those smaller models during post-training.\n• Managing complexity. We make design choices that seek to maximize our
ability to scale the model\ndevelopment process. For example, we opt for a standard dense Transformer model 
architecture (Vaswani\net al., 2017) with minor adaptations, rather than for a mixture-of-experts model (Shazeer et
al., 2017)\nto maximize training stability. Similarly, we adopt a relatively simple post-training procedure 
based\non supervised finetuning (SFT), rejection sampling (RS), and direct preference optimization (DPO;\nRafailov 
et al. (2023)) as opposed to more complex reinforcement learning algorithms (Ouyang et al.,\n2022; Schulman et al.,
2017) that tend to be less stable and harder to scale.\nThe result of our work is Llama 3: a herd of three 
multilingual1 language models with 8B, 70B, and 405B\nparameters. We evaluate the performance of Llama 3 on a 
plethora of benchmark datasets that span a wide\nrange of language understanding tasks. In addition, we perform 
extensive human evaluations that compare\nLlama 3 with competing models. An overview of the performance of the 
flagship Llama 3 model on key\nbenchmarks is presented in Table 2. Our experimental evaluation s

Now let's see if our chat enigine comes up with the same answer.

In [13]:
chat_response = chat_engine.chat("How many new Llama models models are mentioned in the paper?")
print(chat_response.response)
print(chat_response.source_nodes[0])

The paper mentions three new Llama models: Llama 3 8B, Llama 3 70B, and Llama 3 405B.

NodeWithScore(
    node=TextNode(
        id_='7dee73d7-f185-4dd0-84d9-1830c15d286d',
        embedding=None,
        metadata={
            'page_label': '2',
            'file_name': '2407.21783.pdf',
            'file_path': '/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
            'file_type': 'application/pdf',
            'file_size': 9832488,
            'creation_date': '2024-11-14',
            'last_modified_date': '2024-11-14'
        },
        excluded_embed_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        excluded_llm_metadata_keys=[
            'file_name',
            'file_type',
            'file_size',
            'creation_date',
            'last_modified_date',
            'last_accessed_date'
        ],
        relationships={
            <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                node_id='c2aa5e07-1c64-44f6-8701-f3df53873209',
                node_type='4',
                metadata={
                    'page_label': '2',
                    'file_name': '2407.21783.pdf',
                    'file_path': 
'/Users/michaelfrantz/CTME-llm-lecture-resources/prototyping_ai/data/2407.21783.pdf',
                    'file_type': 'application/pdf',
                    'file_size': 9832488,
                    'creation_date': '2024-11-14',
                    'last_modified_date': '2024-11-14'
                },
                hash='2ea90099062b1dd3bee8c1ae15c1a08e55026fc535e90e0d917ed1f6139bfd74'
            )
        },
        metadata_template='{key}: {value}',
        metadata_separator='\n',
        text='Finetuned Multilingual Long context Tool use Release\nLlama 3 8B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 8B 
Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3 70B ✗ ✗ 1 ✗ ✗ April 2024\nLlama 3 70B Instruct ✓ ✗ ✗ ✗ April 2024\nLlama 3.1 
8B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 8B Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 70B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 70B 
Instruct ✓ ✓ ✓ ✓ July 2024\nLlama 3.1 405B ✗ ✓ ✓ ✗ July 2024\nLlama 3.1 405B Instruct ✓ ✓ ✓ ✓ July 2024\nTable 1 
Overview of the Llama 3 Herd of models. All results in this paper are for the Llama 3.1 models.\nscaling laws for 
foundation models, our flagship model outperforms smaller models trained using the\nsame procedure. While our 
scaling laws suggest our flagship model is an approximately compute-optimal\nsize for our training budget, we also 
train our smaller models for much longer than is compute-optimal.\nThe resulting models perform better than 
compute-optimal models at the same inference budget. We\nuse the flagship model to further improve the quality of 
those smaller models during post-training.\n• Managing complexity. We make design choices that seek to maximize our
ability to scale the model\ndevelopment process. For example, we opt for a standard dense Transformer model 
architecture (Vaswani\net al., 2017) with minor adaptations, rather than for a mixture-of-experts model (Shazeer et
al., 2017)\nto maximize training stability. Similarly, we adopt a relatively simple post-training procedure 
based\non supervised finetuning (SFT), rejection sampling (RS), and direct preference optimization (DPO;\nRafailov 
et al. (2023)) as opposed to more complex reinforcement learning algorithms (Ouyang et al.,\n2022; Schulman et al.,
2017) that tend to be less stable and harder to scale.\nThe result of our work is Llama 3: a herd of three 
multilingual1 language models with 8B, 70B, and 405B\nparameters. We evaluate the performance of Llama 3 on a 
plethora of benchmark datasets that span a wide\nrange of language understanding tasks. In addition, we perform 
extensive human evaluations that compare\nLlama 3 with competing models. An overview of the performance of the 
flagship Llama 3 model on key\nbenchmarks is presented in Table 2. Our experimental evaluation s

# Data ingestion

Data often comes in many different formats.
It may come in the form of a PDF, a web page, a code file, etc.
We may need some specific processing pipelines to extract the text from these documents, split them correctly, and vectorize them.

Luckily, LlamaIndex (and other libraries) provide lots of built-in and add-on tools to help you ingest almost any data type.
Instead of loading a PDF, let's load a web page instead.
We will use one of the classes provided by [`llama-index-readers-web`](https://llamahub.ai/l/readers/llama-index-readers-web?from=readers) to load data from a web page.

In this section, we will:
- Load a web page as Markdown
- Split it into chunks following the structured format of the Markdown
- Embed the chunks
- Store the chunks in a vector database
- Create a query engine from the vector database and use it to answer a question


In [14]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


In [15]:
web_docs = SimpleWebPageReader(html_to_text=True).load_data(['https://en.wikipedia.org/wiki/Wikipedia'])

In [16]:
collection = chromadb.EphemeralClient().create_collection("wikipedia", get_or_create=True)
vector_store = ChromaVectorStore(collection)

In [17]:
pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser.from_defaults(),
        embed_model,
    ], 
    vector_store=vector_store
)

In [18]:
nodes = pipeline.run(documents=web_docs)

In [19]:
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [20]:
llm = OpenAI(model="gpt-4o-mini")
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many languages there exactly? Quote the exact text as well.")
print(response.response)

There are currently 339 language editions of Wikipedia.

In [21]:
# Use fuzzywuzzy to find the closest match in the source text
from thefuzz import fuzz, process
# Get the top matching line of text from the source_text_quote
top_match, match_score = process.extractOne(response.response, response.source_nodes[0].text.splitlines(), scorer=fuzz.ratio)
assert top_match in response.source_nodes[0].text
print(f"Quote from source: '{top_match}'")

Quote from source: 'There are currently 339 language editions of Wikipedia (also called _language'

## OCR

There are times where the PDF is not in a format that can be easily read into text.
In these cases, we will need to use optical character recognition (OCR) to convert the images to text.
There are many libraries and cloud services that can do this, but for this example, we will use the `docling` library since our example document is a PDF.
We will then ingest the text into LlamaIndex and use it to answer a question.

![](https://ds4sd.github.io/docling/assets/docling_processing.png)

In [28]:
from llama_index.readers.docling import DoclingReader
from llama_index.node_parser.docling import DoclingNodeParser
from llama_index.core.schema import Document
from llama_index.core import StorageContext

In [23]:
reader = DoclingReader(export_type=DoclingReader.ExportType.JSON)

In [30]:
docling_docs = reader.load_data(pdf_path)

*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*
*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*


In [33]:
len(docling_docs)

1

In [42]:
# We need to set the text template to "{content}" because the default is "{metadata}\n\n{content}",
# and LlamaIndex will try to embed the metadata as well. The metadata is not useful at serach time.
docling_docs[0].text_template
docling_docs[0].text_template = "{content}"

In [43]:
docling_node_parser = DoclingNodeParser()
docling_nodes = docling_node_parser.get_nodes_from_documents(docling_docs)
len(docling_nodes)

710

In [44]:
docling_nodes[0].metadata


{
    'schema_name': 'docling_core.transforms.chunker.DocMeta',
    'version': '1.0.0',
    'doc_items': [
        {
            'self_ref': '#/texts/0',
            'parent': {'$ref': '#/body'},
            'children': [],
            'label': 'page_header',
            'prov': [
                {
                    'page_no': 1,
                    'bbox': {
                        'l': 16.9896183013916,
                        't': 578.7445068359375,
                        'r': 36.339778900146484,
                        'b': 231.99996948242188,
                        'coord_origin': 'BOTTOMLEFT'
                    },
                    'charspan': [0, 38]
                }
            ]
        }
    ],
    'origin': {
        'mimetype': 'application/pdf',
        'binary_hash': 7968363826940540892,
        'filename': '2407.21783.pdf'
    }
}

In [46]:
index = VectorStoreIndex(nodes=docling_nodes, embed_model=embed_model)

In [47]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many new Llama models are mentioned in the paper?")
print(response.response)

The paper mentions three new Llama models: Llama 3.1 8B, Llama 3.1 70B, and Llama 3.1 405B.

In [56]:
print(response.source_nodes[1])

NodeWithScore(
    node=TextNode(
        id_='41152654-030e-4c0b-8e9f-394e0d8705fc',
        embedding=None,
        metadata={
            'schema_name': 'docling_core.transforms.chunker.DocMeta',
            'version': '1.0.0',
            'doc_items': [
                {
                    'self_ref': '#/tables/0',
                    'parent': {'$ref': '#/body'},
                    'children': [],
                    'label': 'table',
                    'prov': [
                        {
                            'page_no': 2,
                            'bbox': {
                                'l': 102.58776092529297,
                                't': 728.816162109375,
                                'r': 508.96331787109375,
                                'b': 585.1917724609375,
                                'coord_origin': 'BOTTOMLEFT'
                            },
                            'charspan': [0, 0]
                        }
                    ]
                }
            ],
            'headings': ['1 Introduction'],
            'captions': [
                'Table 1 Overview of the Llama 3 Herd of models. All results in this paper are for the Llama 3.1 
models.'
            ],
            'origin': {
                'mimetype': 'application/pdf',
                'binary_hash': 7968363826940540892,
                'filename': '2407.21783.pdf'
            }
        },
        excluded_embed_metadata_keys=['schema_name', 'version', 'doc_items', 'origin'],
        excluded_llm_metadata_keys=['schema_name', 'version', 'doc_items', 'origin'],
        relationships={
            <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                node_id='15648112-c57b-4eee-b43f-f324fa4919b1',
                node_type='4',
                metadata={},
                hash='235220e28e49c1ec2c9d9d5336b24cc4aace4ed28a06c1fdc262528cc79c4b5b'
            ),
            <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(
                node_id='9ac0e0d7-79d8-40ff-960f-d9b40b2eacdc',
                node_type='1',
                metadata={
                    'schema_name': 'docling_core.transforms.chunker.DocMeta',
                    'version': '1.0.0',
                    'doc_items': [
                        {
                            'self_ref': '#/texts/17',
                            'parent': {'$ref': '#/body'},
                            'children': [],
                            'label': 'caption',
                            'prov': [
                                {
                                    'page_no': 2,
                                    'bbox': {
                                        'l': 69.73268127441406,
                                        't': 576.0440673828125,
                                        'r': 475.39227294921875,
                                        'b': 565.881103515625,
                                        'coord_origin': 'BOTTOMLEFT'
                                    },
                                    'charspan': [0, 103]
                                }
                            ]
                        }
                    ],
                    'headings': ['1 Introduction'],
                    'origin': {
                        'mimetype': 'application/pdf',
                        'binary_hash': 7968363826940540892,
                        'filename': '2407.21783.pdf'
                    }
                },
                hash='e07333f44f280efdad25ccf4d7bc70e41202f5174cff25208a56f877f0550c8d'
            ),
            <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
                node_id='63fafab9-6b6b-4531-8c83-3461f9dcae41',
                node_type='1',
                metadata={
                    'schema_name': 'docling_core.transforms.chunker.DocMeta',
                    'version': '1.0.0',
                    'doc_items': [
                        {
                            'self_ref': '#/texts/18',
      

# Data splitting

Many times, it's impractical to embed the entire document, and expensive to feed the entire document to the LLM.
Instead, we can split the document into chunks, embed the chunks, and use a retrieval method to find the most relevant chunks.
There are naïve methods that split texts into chunks of a specific length with some overlap;
there are methods that use the structure of the document to split it into sections (e.g. sections, figures, tables);
and there are more advanced methods that use semantic similarity to group the text into chunks.

Since the OCR'd text is just one very long Markdown string, we need to split it into chunks.
One nice way to do that is use the inherent structure of Markdown to split it into sections.
We do that here with LlamaIndex's `MarkdownNodeParser`.

In [62]:
docling_md_docs = DoclingReader(export_type=DoclingReader.ExportType.MARKDOWN).load_data(pdf_path)


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*
*ERR* --- *ERR*
*ERR* Table is not square! *ERR*
*Padding to square...*


In [71]:
from llama_index.core.node_parser import MarkdownNodeParser
from IPython.display import Markdown, display

In [68]:
md_nodes = MarkdownNodeParser.from_defaults().get_nodes_from_documents(documents=docling_md_docs)

In [69]:
len(md_nodes)

105

In [80]:
display(Markdown(md_nodes[9].text))


## 3.2 Model Architecture

Llama 3 uses a standard, dense Transformer architecture (Vaswani et al., 2017). It does not deviate significantly from Llama and Llama 2 (Touvron et al., 2023a,b) in terms of model architecture; our performance gains are primarily driven by improvements in data quality and diversity as well as by increased training scale.

We make a few small modifications compared to Llama 2:

- · We use grouped query attention (GQA; Ainslie et al. (2023)) with 8 key-value heads to improve inference speed and to reduce the size of key-value caches during decoding.
- · We use an attention mask that prevents self-attention between different documents within the same sequence. We find that this change had limited impact during in standard pre-training, but find it to be important in continued pre-training on very long sequences.

Table 3 Overview of the key hyperparameters of Llama 3. We display settings for 8B, 70B, and 405B language models.

|                                                        | 8B             | 70B                   | 405B       |
|--------------------------------------------------------|----------------|-----------------------|------------|
| Layers                                                 | 32             | 80                    | 126        |
| Model Dimension                                        | 4,096          | 8192                  | 16,384     |
| FFN Dimension                                          | 14,336         | 28,672                | 53,248     |
| Attention Heads                                        | 32             | 64                    | 128        |
| Key/Value Heads                                        | 8              | 8                     | 8          |
| Peak Learning Rate Activation Function Vocabulary Size | 3 × 10 - 4     | 1 . 5 × 10 - 4 SwiGLU | 8 × 10 - 5 |
| Positional Embeddings                                  | 128,000 RoPE ( | θ = 500 ,             | 000 )      |

- · We use a vocabulary with 128K tokens. Our token vocabulary combines 100K tokens from the tiktoken 3 tokenizer with 28K additional tokens to better support non-English languages. Compared to the Llama 2 tokenizer, our new tokenizer improves compression rates on a sample of English data from 3.17 to 3.94 characters per token. This enables the model to "read" more text for the same amount of training compute. We also found that adding 28K tokens from select non-English languages improved both compression ratios and downstream performance, with no impact on English tokenization.
- · We increase the RoPE base frequency hyperparameter to 500,000. This enables us to better support longer contexts; Xiong et al. (2023) showed this value to be effective for context lengths up to 32,768.

Llama 3 405B uses an architecture with 126 layers, a token representation dimension of 16,384, and 128 attention heads; see Table 3 for details. This leads to a model size that is approximately compute-optimal according to scaling laws on our data for our training budget of 3 . 8 × 10 25 FLOPs.

In [96]:
md_index = VectorStoreIndex(nodes=md_nodes, embed_model=embed_model)
md_query_engine = md_index.as_query_engine(llm=llm)
response = md_query_engine.query("How many llama3 models are there?")
print(response.response)
print(response.source_nodes[0].text)

There are multiple Llama 3 models, including at least the 8B, 70B, and 405B versions.

## 9.1 Language

Scale. Llama 3 follows the enduring trend of applying straightforward methods at ever increasing scales in 
foundation models. Improvements are driven by increased compute and improved data, with the 405B model using almost
fifty times the pre-training compute budget of Llama 2 70B. Despite containing 405B parameters, our largest Llama 3
in fact contains fewer parameters than earlier and much less performant models such as PALM (Chowdhery et al., 
2023), due to better understanding of scaling laws (Kaplan et al., 2020; Hoffmann et al., 2022). Little is publicly
known about the size of other frontier models, such as Claude 3 or GPT 4 (OpenAI, 2023a), but overall performance 
is compareable.

Small models. Developments in smaller models have paralleled those in large models. Models with fewer parameters 
can dramatically improve inference cost and simplify deployment (Mehta et al., 2024; Team et al., 2024). The 
smaller Llama 3 models achieve this by training far beyond the point of compute optimal training, effectively 
trading training compute for inference efficiency. An alternative path is to distill larger models into smaller 
ones, as in Phi (Abdin et al., 2024).

Architectures. While Llama 3 makes minimal architectural modifiations to compared to Llama 2, other recent 
foundation models have explored other designs. Most notably, mixture of experts architectures (Shazeer et al., 
2017; Lewis et al., 2021; Fedus et al., 2022; Zhou et al., 2022) can be used as an efficient way to increase the 
capacity of a models, such as in Mixtral (Jiang et al., 2024) and Arctic (Snowflake, 2024). Llama 3 outperforms 
these models, suggesting that dense architectures are not the limiting factor, but there remain numerous trade offs
in terms of training and inference efficiency, and model stability at scale.

Open source. Open weights foundation models have rapidly improved over the last year, with Llama3-405B now 
competitive with the current closed weight state-of-the-art. Numerous model families have recently been developed, 
including Mistral (Jiang et al., 2023), Falcon (Almazrouei et al., 2023), MPT (Databricks, 2024), Pythia (Biderman 
et al., 2023), Arctic (Snowflake, 2024), OpenELM (Mehta et al., 2024), OLMo (Groeneveld et al., 2024), StableLM 
(Bellagente et al., 2024), OpenLLaMA (Geng and Liu, 2023), Qwen (Bai et al., 2023), Gemma (Team et al., 2024), Grok
(XAI, 2024), and Phi (Abdin et al., 2024).

Post-training. Post-training Llama 3 follows the established strategy of instruction tuning (Chung et al., 2022; 
Ouyang et al., 2022) followed by alignment with human feedback (Kaufmann et al., 2023). While some studies have 
shown the surprising effectiveness of lightweight alignment procedures (Zhou et al., 2024), Llama 3 uses millions 
of human instructions and preference judgments to improve the pre-trained model, including

techniques such as rejection sampling (Bai et al., 2022), supervised finetuning (Sanh et al., 2022), and Direct 
Preference Optimization (Rafailov et al., 2023). In order to curate these instruction and preference examples, we 
deploy earlier versions of Llama 3 to filter (Liu et al., 2024c), re-write (Pan et al., 2024), or generate prompts 
and responses (Liu et al., 2024b) and apply these techniques through multiple rounds of post-training.

---

# Retrieval

## Starting simple: bm25

BM25 is a simple retrieval method that uses the BM25 algorithm to score the relevance of each document to the query.
The BM25 algorithm is a probabilistic retrieval model that uses the term frequency and inverse document frequency of the query terms to score the relevance of each document.
You should be familiar with the basic idea of tf-idf from your NLP class - you can think of BM25 as a generalization of tf-idf that takes into account more factors.

In [82]:
from llama_index.retrievers.bm25 import BM25Retriever

In [86]:
bm25 = BM25Retriever.from_defaults(nodes=md_nodes)

In [87]:
for node in bm25.retrieve("How many llama3 models are there?"):
    print(f"Score: {node.score:.4f}\nText:\n{node.node.text[:500]}...")

Score: 2.4940
Text:
## 5.4.4 Safety Results

We first highlight Llama 3's general behavior along various axes and then describe results for each specific new 
capability and our effectiveness at mitigating the safety risks.

Overall performance. A comparison of Llama 3's final violation and false refusal rates with similar models can be 
found in Figures 19 and 20. These results focus on our largest parameter size Llama 3 405B model, compared to 
relevant competitors. Two of the competitors are end-to-end systems acce...

Score: 1.5804
Text:
## 5.1.4 Contamination Analysis

We conduct a contamination analysis to estimate to what extent benchmark scores may be influenced by contamination 
of the evaluation data in the pre-training corpus. In previous work, several different contamination methods have 
been used, with various different hyperparameters - we refer to Singh et al. (2024) for an overview. Any of these 
methods can suffer from false positives and negatives, and how to best run contamination analyses is currently 
still an open...

## Dense retrieval (vector search)

BM25 is a simple and fast method that depends on word matching.
But if we want to do more complex retrieval, we can use dense retrieval.
We represent both our query and documents as vectors and use a similarity metric to find the most relevant documents.
This is what's been going on under the hood in the previous examples using `VectorStoreIndex`.

Since most of the mechanics are taken care for us under the hood, let's examine what goes on under the hood.

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt


In [89]:
sample_documents = [
    "My favorite type of dog is a golden retriever.",
    "I like to eat pizza with my friends.",
    "I like to go to the gym in the morning.",
    "I like to play basketball with my friends.",
]

embeddings = np.array(embed_model.get_text_embedding_batch(sample_documents))

query = "What do I like to do with friends?"
query_embedding = np.array(embed_model.get_text_embedding(query))

In [90]:
cosine_similarity(query_embedding.reshape(1, -1), embeddings).squeeze()

array([0.40551496, 0.65599524, 0.45809953, 0.64565545])

Once we have a `VectorStoreIndex`, we can use the `as_retriever` method to get a retriever object.
This uses dense retrieval under the hood, since it already has an embedding model as a part of the class.
Here, we use the `similarity_top_k` parameter to limit the number of results to 2 and show the cosine similarity score along with the beginning of the text.

In [97]:
query = "How many llama3 models are there?"
top_results = md_index.as_retriever(similarity_top_k=2).retrieve(query)
for result in top_results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

Score: 0.7720
Text:
## 9.1 Language

Scale. Llama 3 follows the enduring trend of applying straightforward methods at ever increasing scales in 
foundation models. Improvements are driven by increased compute and improved data, with the 405B model using almost
fifty times the pre-training compute budget of Llama 2 70B. Despite containing 405B parameters, our largest Llama 3
in fact contains fewer parameters than earlier and much less performant models such as PALM (Chowdhery et al., 
2023), due to better understandin...

Score: 0.7673
Text:
## 5.2.5 Math and Reasoning Benchmarks

Our math and reasoning benchmark results are presented in Table 2. Llama 3 8B model outperforms other models of 
similar sizes on GSM8K, MATH, and GPQA. Our 70B model performs significantly better than other models in its class 
on all the benchmarks. Finally, Llama 3 405B model is the best in its category

Table 20 Multilingual benchmarks . For MGSM (Shi et al., 2022), we report 0-shot CoT results for our Llama 3 
models. Multilingual MMLU is an internal ben...

## Hybrid search: query rewriting and reciprocal ranking

Sometimes, you may want several methods of searching over your data, then combining the results.
This is known as hybrid search.

Haveing multiple retrievers may not mean having separate objects - we may just have multiple queries.
In this example, we'll use an LLM to rewrite our query into multiple queries, then use a dense retriever to find the most relevant documents.
Finally, we'll use reciprocal ranking to re-rank the results.


In [98]:
from llama_index.core.retrievers import QueryFusionRetriever

In [101]:
dense_retriever = md_index.as_retriever(similarity_top_k=5)
hybrid_retriever = QueryFusionRetriever(
    [dense_retriever],
    num_queries=3,
    use_async=False,
    mode='reciprocal_rerank',
    verbose=True
)

In [126]:
results = hybrid_retriever.retrieve("How many llama3 models are there?")
for result in results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

Generated queries:
What are the different versions of the Llama3 model?
How many variants of the Llama3 model exist?


Score: 0.0497
Text:
## 9.1 Language

Scale. Llama 3 follows the enduring trend of applying straightforward methods at ever increasing scales in 
foundation models. Improvements are driven by increased compute and improved data, with the 405B model using almost
fifty times the pre-training compute budget of Llama 2 70B. Despite containing 405B parameters, our largest Llama 3
in fact contains fewer parameters than earlier and much less performant models such as PALM (Chowdhery et al., 
2023), due to better understandin...

Score: 0.0489
Text:
## 5.2.5 Math and Reasoning Benchmarks

Our math and reasoning benchmark results are presented in Table 2. Llama 3 8B model outperforms other models of 
similar sizes on GSM8K, MATH, and GPQA. Our 70B model performs significantly better than other models in its class 
on all the benchmarks. Finally, Llama 3 405B model is the best in its category

Table 20 Multilingual benchmarks . For MGSM (Shi et al., 2022), we report 0-shot CoT results for our Llama 3 
models. Multilingual MMLU is an internal ben...

# Reranking

Often, retrieval only gives us a good first pass at finding relevant documents.
We can use re-ranking to improve the results.
There are several re-ranking methods, but in this case we'll use a cross-encoder to re-rank the results.
We'll also introduce a more low-level method for organizing LlamaIndex flows called workflows.

In [130]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.schema import NodeWithScore
from llama_index.core.llms import ChatMessage
from llama_index.core.workflow import (
    Workflow,
    Context,
    step,
    StartEvent,
    StopEvent,
    Event
)
from typing import List


Let's walk throught the code below step by step.
First, we define the events that will be passed between the steps.
This includes a retrieval result, a re-ranking result, and a prompt for the LLM.
The start and stop are already taken care of for us by `StartEvent` and `StopEvent`.

Next, we define our `RAGWithReRank` workflow.
We initialize it with an index and LLM, which we will use for retrieval and answering and the cross-encoder for re-ranking.
We also define exactly what to do for each step of the workflow.
It knows what to do because of the `@step` decorator and the types annotations of each method.

Finally, we can run the workflow with a query.
The workflow will automatically handle passing the events between the steps, so we don't have to worry about that.
We can observe that the results are just what we expect as we saw from the previous examples.
One advantage of this method is that it's much more flexible - you can include arbitrary code, loops, conditionals, etc.

For more detailed documentation on workflows, see [here](https://docs.llamaindex.ai/en/stable/understanding/workflows/).

In [155]:
# Define the events that will be passed between the steps
class RetrievalResult(Event):
    results: List[NodeWithScore]

class ReRankResult(Event):
    results: List[NodeWithScore]

class RagPrompt(Event):
    prompt: str

# Define the workflow
class RAGWithReRank(Workflow):
    # Initialize the workflow with an index and LLM
    def __init__(self, index: VectorStoreIndex = md_index, llm: OpenAI = OpenAI(model="gpt-4o-mini")):
        super().__init__(timeout=10, verbose=False)
        self.index = index
        self.llm = llm
        self.reranker = SentenceTransformerRerank(similarity_top_k=5)

    # The start method is called when the workflow is run.
    # This is the first step of the workflow that takes in a query and returns a retrieval result.
    # Since we want to use a course retriever, we use a top k of 20.
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> RetrievalResult:
        query = ev.query
        await ctx.set('query', query)
        results = self.index.as_retriever(similarity_top_k=20).retrieve(query)
        return RetrievalResult(results=results)
    
    # This step takes the retrieval results and re-ranks them.
    # Notice that the similarity top k is set to 5, so we only keep the top 5 results.
    # This is smaller than the retrieval step since this is about refining a smaller number of results.
    @step
    async def rerank(self, ctx: Context, ev: RetrievalResult) -> ReRankResult:
        results = ev.results
        query = await ctx.get('query')
        reranked = self.reranker.postprocess_nodes(nodes=results, query_str=query)
        return ReRankResult(results=reranked)
    
    # This step creates a prompt for the LLM.
    # It does this by joining the re-ranked results into a single string and formatting it with the query.
    @step
    async def create_prompt(self, ctx: Context, ev: ReRankResult) -> RagPrompt:
        reranked_results = ev.results
        await ctx.set('reranked_results', reranked_results)
        query = await ctx.get('query')
        reranked_str = '\n\n'.join(i.text for i in reranked_results)
        prompt = f"""\
Here is some relevant context:
--------------------------------
{reranked_str}
--------------------------------
Based on the above information and not prior knowledge, please answer the question.
Question:
{query}
"""
        return RagPrompt(prompt=prompt)

    # This step takes the prompt and uses the LLM to answer the question.
    # It also takes the re-ranked results and attaches them to the response in case we want to see the nodes.
    @step
    async def answer(self, ctx: Context, ev: RagPrompt) -> StopEvent:
        prompt = ev.prompt
        ranked_results = await ctx.get('reranked_results')
        messages = [
            ChatMessage.from_str(prompt)
        ]
        answer = await self.llm.achat(messages)
        result = {
            "response": answer,
            "source_nodes": ranked_results
        }
        return StopEvent(result=result)


In [156]:
rag = RAGWithReRank()

In [157]:
await rag.run(query="How many llama3 models are there?")


{
    'response': ChatResponse(
        message=ChatMessage(
            role=<MessageRole.ASSISTANT: 'assistant'>,
            content='Based on the provided information, there are three Llama 3 models mentioned: Llama 3 405B, Llama 3 70B, and Llama 3 8B.',
            additional_kwargs={}
        ),
        raw=ChatCompletion(
            id='chatcmpl-AaG8rHJuwOFEmokijOKLTp1Fb7OLz',
            choices=[
                Choice(
                    finish_reason='stop',
                    index=0,
                    logprobs=None,
                    message=ChatCompletionMessage(
                        content='Based on the provided information, there are three Llama 3 models mentioned: Llama 3 405B, Llama 3 70B, and Llama 3 8B.',
                        refusal=None,
                        role='assistant',
                        audio=None,
                        function_call=None,
                        tool_calls=None
                    )
                )
            ]

In [112]:
class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")


w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()
print(result)

Hello, world!

In [113]:
from llama_index.utils.workflow import draw_all_possible_flows
from IPython.display import HTML

draw_all_possible_flows(MyWorkflow, filename="flow.html")

with open("flow.html", "r") as f:
    display(HTML(f.read()))


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from llama_index.utils.workflow import draw_all_possible_flows                               │
│   2 from IPython.display import HTML                                                             │
│   3                                                                                              │
│   4 draw_all_possible_flows(MyWorkflow, filename="flow.html")                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'llama_index.utils'

# Exercise: Chat with PDF (30 minutes)

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI. If using QueryEngine, use a question and answer format. If using ChatEngine, use a chat format.

If you need a challenge:
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.


In [104]:
import gradio as gr
from tempfile import TemporaryDirectory

In [ ]:
# In the ingest_documents function:
def ingest_documents(file_obj):
    """
    Process an uploaded PDF file and create a query engine for it.
    
    Args:
        file_obj: Gradio file upload object containing the PDF
        
    Returns:
        tuple: (chat_engine, cleared_file_upload, filename)
    """
    if file_obj is None:
        return None, None, ""
        
    with TemporaryDirectory() as temp_dir:
        file_path = os.path.join(temp_dir, 'tmp.pdf')
        # Get the file bytes from the Gradio upload object
        file_bytes = open(file_obj, "rb").read()
        
        with open(file_path, "wb") as f:
            f.write(file_bytes)
            
        documents = SimpleDirectoryReader(temp_dir).load_data()
        llm = OpenAI(model="gpt-4o-mini")
        index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
        chat_engine = index.as_chat_engine(llm=llm)
        
    return chat_engine, None, Path(file_obj.name).name, []

def chat(message, history, chat_engine):
    history.append(
        {
            "role": "user",
            "content": message
        }
    )
    response = chat_engine.chat(message)
    history.append({
        "role": "assistant",
        "content": response.response
    })
    return history, None

# In the Blocks interface:
with gr.Blocks() as demo:
    chat_engine_state = gr.State()  # Renamed for clarity
    gr.Markdown("## RAG Demo: Question answering with a PDF")
    with gr.Row():
        with gr.Column(scale=1):   
            file_upload = gr.File(label="Upload a PDF file", file_types=[".pdf"], file_count="single")
            submit_button = gr.Button("Submit")
            pdf_name = gr.Textbox(label="You are asking about...")
        with gr.Column(scale=5):
            chatbot = gr.Chatbot(label="Chat with the uploaded PDF", type='messages')
            input = gr.Textbox(label="Enter a question", interactive=True)

    # Update to only store the chat_engine in the State
    submit_button.click(
        fn=ingest_documents, 
        inputs=file_upload, 
        outputs=[chat_engine_state, file_upload, pdf_name, chatbot]
    )
    input.submit(
        fn=chat, 
        inputs=[input, chatbot, chat_engine_state], 
        outputs=[chatbot, input]
    )

demo.launch(debug=False)